In [1]:
import sys

PWD = '../../..'
sys.path.append(PWD)

import numpy as np
import pandas as pd

import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import keras_tuner as kt

from src.network_utils import count_params
from src.cross_validator import KerasTunerCrossValidator
from src.dataset import load_dataset_train_test, load_dataset_train_val
from src.models import hp_mlp_builder as hp_model_builder

In [2]:
DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + '/data/tuner'
CROSSVAL_DIR = PWD + '/data/cross_val'

PLANE = 2
CHANNEL = 11

PROJECT_NAME = f'mlp_{PLANE}_{CHANNEL}'

LR = 0.01

N_EPOCHS = 3000
BATCH_SIZE = 2048
MAX_TRIALS = 40
EXECUTIONS_PER_TRIAL = 2

TOP_N = 5
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_base_train, _, y_base_train, _ = load_dataset_train_test(PWD, PLANE, CHANNEL)
X_train, X_val, y_train, y_val = load_dataset_train_val(PWD, PLANE, CHANNEL)

X_base_train.shape, X_train.shape, X_val.shape

((22412, 24), (17929, 24), (4483, 24))

# Model

In [4]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    model = hp_model_builder(hp)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [5]:
model_builder(kt.HyperParameters()).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                600       
                                                                 
 dense_1 (Dense)             (None, 12)                300       
                                                                 
 dense_2 (Dense)             (None, 1)                 13        
                                                                 
Total params: 913
Trainable params: 913
Non-trainable params: 0
_________________________________________________________________


In [6]:
model_callbacks = [
    callbacks.EarlyStopping(patience=50),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=10),
    callbacks.TensorBoard("/tmp/tb_logs/bayesian/" + PROJECT_NAME)
]

# Bayesian tuner

In [7]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR + "/bayesian",
                                         project_name=PROJECT_NAME, overwrite=False)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

# Results

In [8]:
bayesian_tuner.results_summary(1)

Results summary
Results in ../../../data/tuner/bayesian\mlp_2_11
Showing 1 best trials
Trial summary
Hyperparameters:
n_hidden_layers: 1
units_mult: 32
batch_normalization: True
input_batch_normalization: True
dropout: 0.2
Score: 8.616175651550293


In [9]:
best_model = bayesian_tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 24)               96        
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 96)                2400      
                                                                 
 batch_normalization_1 (Batc  (None, 96)               384       
 hNormalization)                                                 
                                                                 
 dropout (Dropout)           (None, 96)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 97        
                                                                 
Total params: 2,977
Trainable params: 2,737
Non-trainabl

---

In [10]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'n_hidden_layers': 1, 'units_mult': 32, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2}
Number of parameters: 2737
========== Model 1 ==========
{'n_hidden_layers': 6, 'units_mult': 8, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2}
Number of parameters: 415873
========== Model 2 ==========
{'n_hidden_layers': 4, 'units_mult': 32, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.5}
Number of parameters: 409969
========== Model 3 ==========
{'n_hidden_layers': 4, 'units_mult': 8, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2}
Number of parameters: 29953
========== Model 4 ==========
{'n_hidden_layers': 1, 'units_mult': 32, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.5}
Number of parameters: 2737


In [11]:
bayesian_tuner.results_summary(TOP_N)

Results summary
Results in ../../../data/tuner/bayesian\mlp_2_11
Showing 5 best trials
Trial summary
Hyperparameters:
n_hidden_layers: 1
units_mult: 32
batch_normalization: True
input_batch_normalization: True
dropout: 0.2
Score: 8.616175651550293
Trial summary
Hyperparameters:
n_hidden_layers: 6
units_mult: 8
batch_normalization: True
input_batch_normalization: True
dropout: 0.2
Score: 8.616738319396973
Trial summary
Hyperparameters:
n_hidden_layers: 4
units_mult: 32
batch_normalization: True
input_batch_normalization: True
dropout: 0.5
Score: 8.62345552444458
Trial summary
Hyperparameters:
n_hidden_layers: 4
units_mult: 8
batch_normalization: True
input_batch_normalization: True
dropout: 0.2
Score: 8.625433921813965
Trial summary
Hyperparameters:
n_hidden_layers: 1
units_mult: 32
batch_normalization: True
input_batch_normalization: True
dropout: 0.5
Score: 8.635278701782227


# Cross-validation for top 5 models

In [12]:
cross_validator = KerasTunerCrossValidator(bayesian_tuner, X_base_train, y_base_train, model_builder,
                                           directory=CROSSVAL_DIR, project_name=PROJECT_NAME,
                                           n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N,
                                           n_cv=CROSSVAL_N_CV, n_executions=CROSSVAL_N_EXEC, overwrite=False)
model_scores = cross_validator()

{'n_hidden_layers': 1, 'units_mult': 32, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2}
Number of parameters: 2737
Got score: 8.8293 (8.6711, 8.9874)
Got score: 8.5013 (8.4773, 8.5253)
Got score: 8.3701 (8.6066, 8.1336)
Got score: 8.4792 (8.5149, 8.4434)
Got score: 9.0057 (8.9365, 9.0748)


{'n_hidden_layers': 6, 'units_mult': 8, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2}
Number of parameters: 415873
Got score: 8.6687 (8.6840, 8.6534)
Got score: 8.3458 (8.3437, 8.3479)
Got score: 8.1702 (8.1905, 8.1500)
Got score: 8.4881 (8.4061, 8.5701)
Got score: 8.7439 (8.7474, 8.7404)


{'n_hidden_layers': 4, 'units_mult': 32, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.5}
Number of parameters: 409969
Got score: 8.6625 (8.6255, 8.6994)
Got score: 8.3671 (8.3632, 8.3710)
Got score: 8.4500 (8.6302, 8.2699)
Got score: 8.4509 (8.4019, 8.5000)
Got score: 8.8499 (8.9026, 8.7972)


{'n_hidden_layers': 4, 'units_mult': 8, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2}
Number of parameters: 29953
Got score: 8.6717 (8.6764, 8.6670)
Got score: 8.3455 (8.3511, 8.3399)
Got score: 8.1547 (8.1805, 8.1288)
Got score: 8.3960 (8.3946, 8.3975)
Got score: 8.7498 (8.7400, 8.7595)


{'n_hidden_layers': 1, 'units_mult': 32, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.5}
Number of parameters: 2737
Got score: 8.7009 (8.7119, 8.6898)
Got score: 8.4356 (8.3942, 8.4770)
Got score: 8.2475 (8.2652, 8.2297)
Got score: 8.6567 (8.6134, 8.7000)
Got score: 8.9669 (8.9988, 8.9351)


In [13]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,8.64,0.24,2737
1,8.48,0.21,415873
2,8.56,0.18,409969
3,8.46,0.22,29953
4,8.60,0.24,2737
